In [1]:
import datetime
import re
from gridstream.protocol import Frame
# from rich.jupyter import print
from rich.console import Console
from rich import inspect
console=Console(tab_size=4,width=300)
print=console.print

paths=['F03B9FFB.txt','F024177B_-_Raw.txt','F03B9FFB-06-22-2021_8-on-demand-meter-reads.txt']
records=[]
for path in paths:
    with open(path,'rt') as f:
        for i,l in enumerate(f):
            if not l.startswith('00FF2A'):
                print(f"discarding line {i} due to header mismatch")
                continue
            try:
                r=re.split('\s+',l,1)
                r[0]=Frame.from_bytes(bytes.fromhex(r[0]),validate=False)
                r[1]=re.sub('\s+',' ',r[1])
                r[1]=datetime.datetime.strptime(r[1].strip(),'%a %b %d %H:%M:%S %Y')
                records.append(r)
            except:
                print(i,l,r)
                raise

len(records)
        

discarding line 581 due to header mismatch

discarding line 1202 due to header mismatch

discarding line 2801 due to header mismatch

discarding line 2814 due to header mismatch

discarding line 2937 due to header mismatch

discarding line 4718 due to header mismatch

discarding line 4936 due to header mismatch

12695

In [2]:
import csv
from decimal import Decimal
class ReadRecord:
    def __init__(self,meter,kh,read_date,read_time,consumption,demand):
        self.meter=meter
        self.kh=float(kh)
        self.timestamp=datetime.datetime.strptime(f"{read_date} {read_time}",'%m/%d/%Y %H:%M:%S')
        self.consumption=Decimal(consumption)*1000 #KWh
        self.demand=float(Decimal(demand)*1000) #KWh
    def __repr__(self):
        return (
            f"{type(self).__name__}({self.meter}, {self.kh:.1f}, {self.timestamp.date()}, "
            f"{self.timestamp.time()}, {self.consumption/1000:.3f}, {self.demand/1000:.3f})"
        )

with open("ground truth.csv") as f:
    reader=csv.reader(f)
    read_header=next(reader)
    read_data=[ReadRecord(*l) for l in reader]
read_data

[ReadRecord(f03b9ffb, 7.2, 2021-03-01, 14:31:08, 37641.426, 79.415),
 ReadRecord(f03b9ffb, 7.2, 2021-03-01, 16:44:40, 37648.598, 37.441),
 ReadRecord(f03b9ffb, 7.2, 2021-03-01, 22:14:01, 37658.906, 47.749),
 ReadRecord(f03b9ffb, 7.2, 2021-03-02, 21:17:07, 37705.090, 44.071),
 ReadRecord(f03b9ffb, 7.2, 2021-03-02, 21:22:58, 37705.234, 44.215),
 ReadRecord(f03b9ffb, 7.2, 2021-06-22, 12:11:15, 46380.824, 153.508),
 ReadRecord(f03b9ffb, 7.2, 2021-06-22, 12:39:18, 46383.801, 156.485),
 ReadRecord(f03b9ffb, 7.2, 2021-06-22, 13:40:07, 46390.645, 163.329),
 ReadRecord(f03b9ffb, 7.2, 2021-06-22, 14:11:33, 46394.801, 167.485),
 ReadRecord(f03b9ffb, 7.2, 2021-06-22, 14:45:25, 46399.168, 171.852),
 ReadRecord(f03b9ffb, 7.2, 2021-06-22, 16:21:49, 46414.340, 62.981),
 ReadRecord(f03b9ffb, 7.2, 2021-06-22, 17:09:42, 46423.031, 71.672),
 ReadRecord(f03b9ffb, 7.2, 2021-06-22, 20:31:41, 46448.680, 97.321),
 ReadRecord(f03b9ffb, 7.2, 2021-06-23, 00:17:52, 46464.125, 112.766),
 ReadRecord(f03b9ffb, 7.2, 2

In [5]:
from collections import Counter
from rich.table import Table
from rich.progress import track
from gridstream.protocol.frame import DataFrame,RoutingFrame

def counter_rich_table(counter,console,options):
    table=Table('Key','Occurences')
    for k in sorted(counter):
        table.add_row(str(k),str(counter[k]))
    yield(table)
Counter.__rich_console__=counter_rich_table

def get_packets(filter):
    return sorted([r for r in track(records) if filter(*r)],key=lambda r:r[1])

def display_packets(packets):
    table=Table('Timestamp','type','type (bits)',"From","To","Undecoded bytes")
    for r,t in packets:
        table.add_row(
            str(t),
            f"0x{r.packet.packet_type:02X}",
            f"{r.packet.packet_type:08b}",
            r.packet.source.hex(),
            r.packet.destination.hex(),
            r.packet.bytes[r.packet._fields[-1].slice].hex()
        )
    print(table)


In [8]:
candidates=get_packets(lambda r,t: isinstance(r,DataFrame) and any([(d.timestamp>t) and (d.timestamp-t)<=datetime.timedelta(seconds=60) for d in read_data]))
print(len(candidates))
display_packets(candidates)


Output()

16

┏━━━━━━━━━━━━━━━━━━━━━┳━━━━━━┳━━━━━━━━━━━━━┳━━━━━━━━━━┳━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┓
┃ Timestamp           ┃ type ┃ type (bits) ┃ From     ┃ To       ┃ Undecoded bytes                      ┃
┡━━━━━━━━━━━━━━━━━━━━━╇━━━━━━╇━━━━━━━━━━━━━╇━━━━━━━━━━╇━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┩
│ 2021-03-01 14:30:46 │ 0x29 │ 00101001    │ f03b9ffb │ f0611c3e │ 399106009150fd3303ef1620             │
│ 2021-03-01 16:44:17 │ 0x29 │ 00101001    │ f03b9ffb │ f0611c3e │ 729830e31680                         │
│ 2021-03-01 22:13:25 │ 0x29 │ 00101001    │ f03b9ffb │ f0611c3e │ 3d91060091504cad13001700             │
│ 2021-03-01 22:13:25 │ 0x29 │ 00101001    │ f03b9ffb │ f0fbbc6f │ 4d9101001f6c04060091504cad133c1d40   │
│ 2021-03-01 22:13:25 │ 0x29 │ 00101001    │ f03b9ffb │ f0611c3e │ 5191080091504cad0129bd44815213501740 │
│ 2021-03-02 21:16:34 │ 0x29 │ 00101001    │ f03b9ffb │ f091461c │ c991060091505d42122e2980             │
│ 2021-03-02 21:22:43 │ 0x29 │ 00101001    │ f03b9ffb │ f091461c │ 999106008f50b0001eb32970             │
│ 2021-06-22 12:11:02 │ 0x29 │ 00101001    │ f15df2b3 │ f03b9ffb │ 909101002532050600bc50e04611693f40   │
│ 2021-06-22 12:38:56 │ 0x29 │ 00101001    │ f03b9ffb │ f15df2b3 │ 039106009250e5453e743f30             │
│ 2021-06-22 13:39:54 │ 0x29 │ 00101001    │ f03b9ffb │ 40d0d00a │ c591010026320506009150e4982f7d2130   │
│ 2021-06-22 14:11:17 │ 0x29 │ 00101001    │ f03b9ffb │ 40d0d00a │ a59106009250d3b03d062140             │
│ 2021-06-22 14:44:58 │ 0x29 │ 00101001    │ f03b9ffb │ 40d0d00a │ dd910600925025f33f2e2100             │
│ 2021-06-22 16:21:13 │ 0x29 │ 00101001    │ f15df2b3 │ f03b9ffb │ 609101001f6c040600bc50499828783f70   │
│ 2021-06-22 16:21:20 │ 0x29 │ 00101001    │ f03b9ffb │ f02e6589 │ 1b910100246c050600925098bc18d63250   │
│ 2021-06-22 17:09:15 │ 0x29 │ 00101001    │ f15df2b3 │ f03b9ffb │ 04910600bc50867932ef3f80             │
│ 2021-06-22 20:31:18 │ 0x29 │ 00101001    │ f03b9ffb │ f1366742 │ 17910600925068572abe3f20             │
└─────────────────────┴──────┴─────────────┴──────────┴──────────┴──────────────────────────────────────┘

In [3]:
from gridstream.protocol import Frame

frame=records[0][0]

frame


In [4]:
frame.data_bytes.hex()

'30ffffffffffff50cfb5d9e40070000c3582a483f03b9ffb01001e320414c37e90'

In [5]:
frame.packet

In [6]:
frame.packet.timing

5315

In [7]:
packet=frame.packet
cls=packet.__class__
packet.bytes[cls.timing.end:].hex(),packet.bytes.hex()

('7e90', '30ffffffffffff50cfb5d9e40070000c3582a483f03b9ffb01001e320414c37e90')

In [8]:
print(packet)

ARPPacket Packet contains 33 bytes
┏━━━━━━━━┳━━━━━━━━━━━━━┳━━━━━━━━━━━━┳━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┓
┃ Offset ┃ Field Name  ┃ Byte range ┃ Bytes        ┃ Value                       ┃
┡━━━━━━━━╇━━━━━━━━━━━━━╇━━━━━━━━━━━━╇━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┩
│   0    │ packet_type │ 0-1        │ 30           │ 48                          │
│   1    │ destination │ 1-7        │ ffffffffffff │ b'\xff\xff\xff\xff\xff\xff' │
│   7    │ source      │ 7-13       │ 50cfb5d9e400 │ b'P\xcf\xb5\xd9\xe4\x00'    │
│  13    │ sequence    │ 13-14      │ 70           │ 112                         │
│  14    │ uptime      │ 14-18      │ 000c3582     │ 800130                      │
│  18    │ Unknown     │ 18-20      │ a483         │ None                        │
│  20    │ lan_address │ 20-24      │ f03b9ffb     │ b'\xf0;\x9f\xfb'            │
│  24    │ Unknown     │ 24-29      │ 01001e3204   │ None                        │
│  29    │ timing      │ 29-31      │ 14c3         │ 5315                        │
│  31    │ tail        │ 31-None    │ 7e90         │ None                        │
└────────┴─────────────┴────────────┴──────────────┴─────────────────────────────┘

In [9]:
f=Frame.from_bytes(bytes.fromhex("00FF2AD5001729F0611C3EF03B9FFB399106009150FD3303EF16207458"))

In [10]:
packet=f.packet

In [11]:
print(packet)

DataPacket Packet contains 21 bytes
┏━━━━━━━━┳━━━━━━━━━━━━━┳━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━┓
┃ Offset ┃ Field Name  ┃ Byte range ┃ Bytes                    ┃ Value            ┃
┡━━━━━━━━╇━━━━━━━━━━━━━╇━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━┩
│   0    │ packet_type │ 0-1        │ 29                       │ 41               │
│   1    │ destination │ 1-5        │ f0611c3e                 │ b'\xf0a\x1c>'    │
│   5    │ source      │ 5-9        │ f03b9ffb                 │ b'\xf0;\x9f\xfb' │
│   9    │ tail        │ 9-None     │ 399106009150fd3303ef1620 │ None             │
└────────┴─────────────┴────────────┴──────────────────────────┴──────────────────┘

In [12]:
from gridstream.protocol.frame import DataFrame
len([r for r in records if isinstance(r[0],DataFrame)])

951

In [14]:
print('Record Lengths')
c=Counter([r[0].length for r in records if isinstance(r[0],DataFrame)])
print(c)


Record Lengths

┏━━━━━┳━━━━━━━━━━━━┓
┃ Key ┃ Occurences ┃
┡━━━━━╇━━━━━━━━━━━━┩
│ 15  │ 36         │
│ 20  │ 224        │
│ 21  │ 304        │
│ 26  │ 36         │
│ 27  │ 5          │
│ 32  │ 3          │
│ 47  │ 98         │
│ 69  │ 31         │
│ 115 │ 2          │
│ 231 │ 2          │
│ 251 │ 210        │
└─────┴────────────┘

In [15]:
print('Subtype 0x29 Record Lengths')
print(Counter([r[0].length for r in records if isinstance(r[0],DataFrame) and r[0].packet.packet_type==0x29]))

Subtype 0x29 Record Lengths

┏━━━━━┳━━━━━━━━━━━━┓
┃ Key ┃ Occurences ┃
┡━━━━━╇━━━━━━━━━━━━┩
│ 15  │ 20         │
│ 20  │ 49         │
│ 21  │ 194        │
│ 26  │ 35         │
│ 27  │ 5          │
│ 32  │ 3          │
└─────┴────────────┘

In [16]:
def get_packets(filter):
    return sorted([r for r in records if filter(*r)],key=lambda r:r[1])

def display_packets(packets):
    table=Table('Timestamp','type','type (bits)',"From","To","Undecoded bytes")
    for r,t in packets:
        table.add_row(
            str(t),
            f"0x{r.packet.packet_type:02X}",
            f"{r.packet.packet_type:08b}",
            r.packet.source.hex(),
            r.packet.destination.hex(),
            r.packet.bytes[r.packet._fields[-1].slice].hex()
        )
    print(table)

display_packets(get_packets(lambda r,t:isinstance(r,DataFrame) and r.length==21 and r.packet.bytes[10]&0xF0==0x90))


┏━━━━━━━━━━━━━━━━━━━━━┳━━━━━━┳━━━━━━━━━━━━━┳━━━━━━━━━━┳━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━━━┓
┃ Timestamp           ┃ type ┃ type (bits) ┃ From     ┃ To       ┃ Undecoded bytes          ┃
┡━━━━━━━━━━━━━━━━━━━━━╇━━━━━━╇━━━━━━━━━━━━━╇━━━━━━━━━━╇━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━━━┩
│ 2021-03-01 09:55:38 │ 0x29 │ 00101001    │ f03b9ffb │ f0611c3e │ 819006009350027b34581600 │
│ 2021-03-01 11:14:22 │ 0x21 │ 00100001    │ f03b9ffb │ f0611c3e │ 139006009b504bef3c201670 │
│ 2021-03-01 11:30:45 │ 0x29 │ 00101001    │ f03b9ffb │ f0611c3e │ 259006008950cbf032541670 │
│ 2021-03-01 12:14:13 │ 0x29 │ 00101001    │ f03b9ffb │ f0611c3e │ 459006009f50964113111620 │
│ 2021-03-01 12:35:28 │ 0x29 │ 00101001    │ f03b9ffb │ f0611c3e │ e59006009750af5239d21670 │
│ 2021-03-01 12:59:07 │ 0x29 │ 00101001    │ f03b9ffb │ f0611c3e │ 159006009d50d51515761660 │
│ 2021-03-01 13:21:14 │ 0x21 │ 00100001    │ f03b9ffb │ f0611c3e │ af9006009150fa0e0edf1670 │
│ 2021-03-01 13:39:49 │ 0x29 │ 00101001    │ f03b9ffb │ f0611c3e │ 1190060097508be338851690 │
│ 2021-03-01 13:57:45 │ 0x29 │ 00101001    │ f03b9ffb │ f0611c3e │ 279006009b508a5111741670 │
│ 2021-03-01 14:30:46 │ 0x29 │ 00101001    │ f03b9ffb │ f0611c3e │ 399106009150fd3303ef1620 │
│ 2021-03-01 19:14:22 │ 0x29 │ 00101001    │ f03b9ffb │ f0611c3e │ 23900600a350815d16991610 │
│ 2021-03-01 19:28:39 │ 0x29 │ 00101001    │ f03b9ffb │ f0611c3e │ e19006008850e8df1d521600 │
│ 2021-03-01 22:13:25 │ 0x29 │ 00101001    │ f03b9ffb │ f0611c3e │ 3d91060091504cad13001700 │
│ 2021-03-02 01:39:48 │ 0x29 │ 00101001    │ f03b9ffb │ f0bcd31d │ 3990060091507f0200113f10 │
│ 2021-03-02 03:14:12 │ 0x29 │ 00101001    │ f03b9ffb │ 40d0d00a │ 8b900600ee50d8592eeb2560 │
│ 2021-03-02 04:14:13 │ 0x29 │ 00101001    │ f03b9ffb │ 40d0d00a │ e990060093509b2e09dc2500 │
│ 2021-03-02 04:28:31 │ 0x29 │ 00101001    │ f03b9ffb │ 40d0d00a │ 919006007d5085c510d62530 │
│ 2021-03-02 05:52:57 │ 0x29 │ 00101001    │ f03b9ffb │ 40d0d00a │ 8f9006007b500deb1adb2560 │
│ 2021-03-02 07:28:39 │ 0x29 │ 00101001    │ f03b9ffb │ 40d0d00a │ 3590060088501bcb26ab2590 │
│ 2021-03-02 08:14:13 │ 0x29 │ 00101001    │ f03b9ffb │ 40d0d00a │ 9d9006009550632338aa2510 │
│ 2021-03-02 18:06:22 │ 0x29 │ 00101001    │ f03b9ffb │ f091461c │ d59006009a502a0916b12950 │
│ 2021-03-02 19:05:28 │ 0x29 │ 00101001    │ f03b9ffb │ f091461c │ eb9006009850cddb1d9b2930 │
│ 2021-03-02 19:14:22 │ 0x21 │ 00100001    │ f03b9ffb │ f091461c │ 119006009c500748295c2980 │
│ 2021-03-02 19:19:45 │ 0x29 │ 00101001    │ f03b9ffb │ f091461c │ bf9006008850b8f5245c2950 │
│ 2021-03-02 19:28:39 │ 0x29 │ 00101001    │ f03b9ffb │ f091461c │ d39006008850a43b30062940 │
│ 2021-03-02 19:38:05 │ 0x29 │ 00101001    │ f03b9ffb │ f091461c │ 039006008f50e9a1067c2930 │
│ 2021-03-02 19:52:24 │ 0x29 │ 00101001    │ f03b9ffb │ f091461c │ bb900600795078d00e002960 │
│ 2021-03-02 21:16:34 │ 0x29 │ 00101001    │ f03b9ffb │ f091461c │ c991060091505d42122e2980 │
│ 2021-03-02 21:22:43 │ 0x29 │ 00101001    │ f03b9ffb │ f091461c │ 999106008f50b0001eb32970 │
│ 2021-03-03 05:55:11 │ 0x29 │ 00101001    │ f03b9ffb │ f091461c │ af9006009450f58320692970 │
│ 2021-03-03 06:09:28 │ 0x29 │ 00101001    │ f03b9ffb │ f091461c │ 679006008850bffd27282960 │
│ 2021-06-15 02:05:26 │ 0x29 │ 00101001    │ f024177b │ 908a8a61 │ 1f900600c450855f05223f80 │
│ 2021-06-18 16:02:52 │ 0x29 │ 00101001    │ f024177b │ f0ef6c34 │ c2950600f150b31001123f90 │
│ 2021-06-18 18:54:23 │ 0x29 │ 00101001    │ f06ebc01 │ f024177b │ 96910600f150ece323152060 │
│ 2021-06-18 19:01:34 │ 0x29 │ 00101001    │ f024177b │ f0f28ef7 │ 74950600f1508c5436e23580 │
│ 2021-06-22 12:38:56 │ 0x29 │ 00101001    │ f03b9ffb │ f15df2b3 │ 039106009250e5453e743f30 │
│ 2021-06-22 14:11:17 │ 0x29 │ 00101001    │ f03b9ffb │ 40d0d00a │ a59106009250d3b03d062140 │
│ 2021-06-22 14:44:58 │ 0x29 │ 00101001    │ f03b9ffb │ 40d0d00a │ dd910600925025f33f2e2100 │
│ 2021-06-22 15:38:56 │ 0x29 │ 00101001    │ f06ebc01 │ f03b9ffb │ 72910600bc50782939303f20 │
│ 2021-06-22 17:09:15 │ 0x29 │ 00101001    │ f15df2b

In [17]:
target=bytes.fromhex("f03b9ffb")
display_packets(get_packets(lambda r,t:isinstance(r,DataFrame) and (r.packet.source==target or r.packet.destination==target)))


┏━━━━━━━━━━━━━━━━━━━━━┳━━━━━━┳━━━━━━━━━━━━━┳━━━━━━━━━━┳━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┓
┃ Timestamp           ┃ type ┃ type (bits) ┃ From     ┃ To       ┃ Undecoded bytes                                                                                                                                                                                                                         ┃
┡━━━━━━━━━━━━━━━━━━━━━╇━━━━━━╇━━━━━━━━━━━━━╇━━━━━━━━━━╇━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┩
│ 2021-03-01 08:43:20 │ 0x29 │ 00101001    │ f03b9ffb │ f0611c3e │ ae9840141630                                                                                                                                                                                                                            │
│ 2021-03-01 08:43:36 │ 0x29 │ 00101001    │ f03b9ffb │ f0611c3e │ ba9804b51650                                                                                                                                                                                                                            │
│ 2021-03-01 08:43:45 │ 0x29 │ 00101001    │ f03b9ffb │ f0611c3e │ ca9808441600                                                                                                                                                                                                                            │
│ 2021-03-01 09:39:48 │ 0x29 │ 00101001    │ f03b9ffb │ f0611c3e │ a19001001e320406009950d1ca097e1620                                                                                                                                                                                                      │
│ 2021-03-01 09:55:38 │ 0x29 │ 00101001    │ f03b9ffb │ f0611c3e │ 819006009350027b34581600                                                                                                                                                                                                                │
│ 2021-03-01 11:14:22 │ 0x21 │ 00100001    │ f03b9ffb │ f0611c3e │ 139006009b504bef3c201670                                                                                                                                                                                                                │
│ 2021-03-01 11:30:45 │ 0x29 │ 00101001    │ f03b9ffb │ f0611c3e │ 259006008950cbf032541670                                                                                                                                                                                                                │
│ 2021-03-01 11:50:10 │ 0x21 │ 00100001    │ f03b9ffb │ f0653657 │ 728801001f32042e241140                                                                                                                                                                                                                  │
│ 2021-03-01 11:50:10 │ 0x51 │ 01010001    │ f03b9ffb │ f0653657 │ 74603d295200029892000c61bba483010150cfb5d9e400f03b9ffb00010324060203070507070500000002a3c8001f320401f120302081802e2c1100                                                                                                                │
│ 2021-03-01 12:14:13 │ 0x29 │ 00101001    │ f03b9ffb │ f0611c3e │ 459006009f50964113111620                                                                                                                                                                                                                │
│ 2021-03-01 12:35:28 │ 0x29 │ 00101001    │ f03b9ffb │ f0611c3e │ e59006009750af5239d2

In [18]:
print('Record types')
print(Counter([f"{r[0].packet.packet_type:0x}" for r in records if isinstance(r[0],DataFrame)]))

Record types

┏━━━━━┳━━━━━━━━━━━━┓
┃ Key ┃ Occurences ┃
┡━━━━━╇━━━━━━━━━━━━┩
│ 21  │ 299        │
│ 22  │ 3          │
│ 29  │ 306        │
│ 51  │ 27         │
│ 55  │ 4          │
│ c0  │ 312        │
└─────┴────────────┘

In [19]:
print('Record types')
print(Counter([f"{r[0].packet.packet_type:08b}" for r in records if isinstance(r[0],DataFrame)]))

Record types

┏━━━━━━━━━━┳━━━━━━━━━━━━┓
┃ Key      ┃ Occurences ┃
┡━━━━━━━━━━╇━━━━━━━━━━━━┩
│ 00100001 │ 299        │
│ 00100010 │ 3          │
│ 00101001 │ 306        │
│ 01010001 │ 27         │
│ 01010101 │ 4          │
│ 11000000 │ 312        │
└──────────┴────────────┘

In [20]:
print('Record Sources')
print(Counter([r[0].packet.source.hex() for r in records if isinstance(r[0],DataFrame)]))

Record Sources

┏━━━━━━━━━━┳━━━━━━━━━━━━┓
┃ Key      ┃ Occurences ┃
┡━━━━━━━━━━╇━━━━━━━━━━━━┩
│ f024177b │ 498        │
│ f02e6589 │ 1          │
│ f03b9ffb │ 372        │
│ f03d4cd7 │ 28         │
│ f0469f5a │ 2          │
│ f04cf3cc │ 1          │
│ f05191b0 │ 1          │
│ f05dd3fb │ 1          │
│ f0625edc │ 3          │
│ f0625f05 │ 1          │
│ f06ebc01 │ 3          │
│ f0b0ca58 │ 1          │
│ f0b46d99 │ 3          │
│ f0bcd31d │ 1          │
│ f0bf21f2 │ 2          │
│ f0ec41a2 │ 4          │
│ f0ee9d21 │ 1          │
│ f0f28d56 │ 4          │
│ f0f28ef7 │ 2          │
│ f0f4eb8d │ 1          │
│ f0fa4a66 │ 4          │
│ f0fc4db1 │ 1          │
│ f12056a2 │ 1          │
│ f1366742 │ 3          │
│ f14c7561 │ 4          │
│ f14d9972 │ 2          │
│ f15d58a5 │ 3          │
│ f15df2b3 │ 3          │
└──────────┴────────────┘

In [21]:
print('Record Destinations')
print(Counter([r[0].packet.destination.hex() for r in records if isinstance(r[0],DataFrame)]))

Record Destinations

┏━━━━━━━━━━┳━━━━━━━━━━━━┓
┃ Key      ┃ Occurences ┃
┡━━━━━━━━━━╇━━━━━━━━━━━━┩
│ 40c824f5 │ 3          │
│ 40d0d00a │ 18         │
│ 500c7a8a │ 1          │
│ 500c8422 │ 1          │
│ 8073ab83 │ 67         │
│ 8073ad6e │ 24         │
│ 8073ae67 │ 49         │
│ 8073aeac │ 18         │
│ 8073aeae │ 52         │
│ 908a8a61 │ 8          │
│ 908f8a38 │ 2          │
│ 90b5f1b1 │ 1          │
│ f01a2a35 │ 1          │
│ f024177b │ 76         │
│ f02e1841 │ 6          │
│ f02e5d7d │ 1          │
│ f02e6589 │ 1          │
│ f02e65e8 │ 1          │
│ f02e6611 │ 1          │
│ f02e66bb │ 1          │
│ f02e66d3 │ 1          │
│ f02eaebf │ 1          │
│ f02eb75e │ 1          │
│ f02ec34a │ 1          │
│ f02ee2e6 │ 1          │
│ f02ee5ec │ 2          │
│ f03a9b6c │ 1          │
│ f03a9ed7 │ 1          │
│ f03b9ffb │ 11         │
│ f04996ce │ 1          │
│ f04dd4c6 │ 1          │
│ f04e6b5f │ 1          │
│ f04fbee7 │ 2          │
│ f050307a │ 1          │
│ f05064cd │ 2          │
│ f05086d0 │ 1          │
│ f05191b0 │ 1          │
│ f0519282 │ 1          │
│ f0519289 │ 1          │
│ f05361e7 │ 1          │
│ f0536291 │ 1          │
│ f05395e4 │ 2          │
│ f053eb19 │ 1          │
│ f0542f4a │ 1          │
│ f054d3bf │ 2          │
│ f054d4c0 │ 1          │
│ f05644ad │ 1          │
│ f057af33 │ 8          │
│ f059caea │ 2          │
│ f05a4c4d │ 1          │
│ f05c3966 │ 3          │
│ f05dd34d │ 1          │
│ f05fc165 │ 2          │
│ f05fcb84 │ 1          │
│ f05fe73e │ 1          │
│ f0611c3e │ 29         │
│ f0613066 │ 1          │
│ f0620093 │ 1          │
│ f0625edc │ 4          │
│ f062e3cd │ 2          │
│ f06313ee │ 3          │
│ f063cf35 │ 2          │
│ f0649dfa │ 1          │
│ f0653657 │ 2          │
│ f06ebc01 │ 2          │
│ f072fea5 │ 1          │
│ f07c4eed │ 1          │
│ f07c5f0c │ 2          │
│ f091461c │ 17         │
│ f09e3c0f │ 1          │
│ f0a934c9 │ 1          │
│ f0acf320 │ 1          │
│ f0aeb950 │ 2          │
│ f0b0047f │ 1          │
│ f0b094cb │ 1          │
│ f0b0ca58 │ 1          │
│ f0b0d132 │ 1          │
│ f0b44b4d │ 1          │
│ f0b47373 │ 3          │
│ f0b47389 │ 1          │
│ f0b4894d │ 1          │
│ f0b4897b │ 1          │
│ f0b4ceda │ 1          │
│ f0bcd31d │ 4          │
│ f0beaf64 │ 1          │
│ f0bf21f2 │ 3          │
│ f0bff9b4 │ 2          │
│ f0bffa9e │ 1          │
│ f0c72bd0 │ 1          │
│ f0e68017 │ 1          │
│ f0eae3c2 │ 2          │
│ f0eae3e0 │ 3          │
│ f0ebb876 │ 1          │
│ f0ebfd4a │ 2          │
│ f0ec41f0 │ 1          │
│ f0ee9d21 │ 2          │
│ f0ef25d9 │ 2          │
│ f0ef3bce │ 2          │
│ f0ef53f6 │ 1          │
│ f0ef6c34 │ 2          │
│ f0f28ef7 │ 3          │
│ f0f3ccdb │ 1          │
│ f0f5c67e │ 3          │
│ f0f8266f │ 1          │
│ f0f8e2e9 │ 2          │
│ f0f8e378 │ 1          │
│ f0f95f1c │ 2          │
│ f0fbbc6f │ 2          │
│ f0fd5a92 │ 1          │
│ f0fe903a │ 1          │
│ f0feb28f │ 2          │
│ f0fed768 │ 1          │
│ f10676d2 │ 1          │
│ f10679de │ 3          │
│ f106ac5d │ 1          │
│ f115ed71 │ 2          │
│ f118fed8 │ 1          │
│ f11ac82c │ 1          │
│ f11e59d6 │ 1          │
│ f11e6072 │ 2          │
│ f11e6a6b │ 1          │
│ f1227563 │ 2          │
│ f1337349 │ 5          │
│ f133c5a4 │ 11         │
│ f133e077 │ 1          │
│ f1366742 │ 3          │
│ f136725b │ 1          │
│ f13d8d99 │ 3          │
│ f1419add │ 1          │
│ f141a241 │ 1          │
│ f141d4a3 │ 2          │
│ f1459b3b │ 2          │
│ f14af328 │ 3          │
│ f14c7561 │ 1          │
│ f14d9972 │ 1          │
│ f14dd89e │ 1          │
│ f15725f9 │ 1          │
│ f157a2a1 │ 8          │
│ f157bb38 │ 1          │
│ f159147b │ 2          │
│ f159beb3 │ 1          │
│ f15cade4 │ 1          │
│ f15d58a5 │ 1          │
│ f15df2b3 │ 3          │
│ f15e88cf │ 17         │
│ f15f2e2d │ 1          │
│ f162bf77 │ 21         │
│ f165f3cb │ 1          │
│ f16613cb │ 1          │
│ ffffffff │ 312        │
└──────────┴──────────

In [22]:
for n in (0xD5,0X55):
    print(f"{n:3d}",f"{n:08b}")

213 11010101

85 01010101

In [23]:
0x11

17

In [24]:
import struct
def f(r,t):
#     if t.month!=6:
#         return False
    if not isinstance(r,DataFrame):
        return False
    if r.packet.source!=target:# and r.packet.destination!=target:
        return False
    if r.packet.bytes[10]!=0x91:
        return False
    return r.packet.bytes.find(b'\x06\x00')==11
packets=get_packets(f)
display_packets(packets)

┏━━━━━━━━━━━━━━━━━━━━━┳━━━━━━┳━━━━━━━━━━━━━┳━━━━━━━━━━┳━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━━━┓
┃ Timestamp           ┃ type ┃ type (bits) ┃ From     ┃ To       ┃ Undecoded bytes          ┃
┡━━━━━━━━━━━━━━━━━━━━━╇━━━━━━╇━━━━━━━━━━━━━╇━━━━━━━━━━╇━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━━━┩
│ 2021-03-01 14:30:46 │ 0x29 │ 00101001    │ f03b9ffb │ f0611c3e │ 399106009150fd3303ef1620 │
│ 2021-03-01 22:13:25 │ 0x29 │ 00101001    │ f03b9ffb │ f0611c3e │ 3d91060091504cad13001700 │
│ 2021-03-02 21:16:34 │ 0x29 │ 00101001    │ f03b9ffb │ f091461c │ c991060091505d42122e2980 │
│ 2021-03-02 21:22:43 │ 0x29 │ 00101001    │ f03b9ffb │ f091461c │ 999106008f50b0001eb32970 │
│ 2021-06-22 12:38:56 │ 0x29 │ 00101001    │ f03b9ffb │ f15df2b3 │ 039106009250e5453e743f30 │
│ 2021-06-22 14:11:17 │ 0x29 │ 00101001    │ f03b9ffb │ 40d0d00a │ a59106009250d3b03d062140 │
│ 2021-06-22 14:44:58 │ 0x29 │ 00101001    │ f03b9ffb │ 40d0d00a │ dd910600925025f33f2e2100 │
│ 2021-06-22 20:31:18 │ 0x29 │ 00101001    │ f03b9ffb │ f1366742 │ 17910600925068572abe3f20 │
└─────────────────────┴──────┴─────────────┴──────────┴──────────┴──────────────────────────┘

In [25]:
consumptions=[37641426,37658906,37705090,37705234,46383801,46394801,46399168,46448680]
kh=[c/7.2 for c in consumptions]
[int(v) for v in kh]
demands=[79.415,47.749,44.071,44.215,156.485,167.485,171.852,97.321]

In [26]:
expected=kh
type_names={'h':'short','H':'ushort','i':'int','I':'uint'}

for code in 'hHiI':
    fmt=f'!{code}'
    size=struct.calcsize(fmt)
    for offset in range(13,19):
        print(f"{type_names[code]} at {offset}")
        table=Table("Bytes","Value","Expected","Factor")
        for (r,t),e in zip(packets,expected):
            packet=r.packet
            try:
                value=struct.unpack_from(fmt,packet.bytes,offset=offset)[0]
                table.add_row(packet.bytes[offset:offset+size].hex(),str(value),f"{e!r:10s}",f"{e/value:10.4f}")
            except:
                table.add_row(packet.bytes[offset:offset+size].hex(),"-","-","-")
            
        print(table)
            

short at 13

┏━━━━━━━┳━━━━━━━━┳━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━┓
┃ Bytes ┃ Value  ┃ Expected          ┃ Factor     ┃
┡━━━━━━━╇━━━━━━━━╇━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━┩
│ 9150  │ -28336 │ 5227975.833333333 │  -184.4994 │
│ 9150  │ -28336 │ 5230403.611111111 │  -184.5851 │
│ 9150  │ -28336 │ 5236818.055555555 │  -184.8115 │
│ 8f50  │ -28848 │ 5236838.055555555 │  -181.5321 │
│ 9250  │ -28080 │ 6442194.583333333 │  -229.4229 │
│ 9250  │ -28080 │ 6443722.361111111 │  -229.4773 │
│ 9250  │ -28080 │ 6444328.888888889 │  -229.4989 │
│ 9250  │ -28080 │ 6451205.555555555 │  -229.7438 │
└───────┴────────┴───────────────────┴────────────┘

short at 14

┏━━━━━━━┳━━━━━━━┳━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━┓
┃ Bytes ┃ Value ┃ Expected          ┃ Factor     ┃
┡━━━━━━━╇━━━━━━━╇━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━┩
│ 50fd  │ 20733 │ 5227975.833333333 │   252.1572 │
│ 504c  │ 20556 │ 5230403.611111111 │   254.4466 │
│ 505d  │ 20573 │ 5236818.055555555 │   254.5481 │
│ 50b0  │ 20656 │ 5236838.055555555 │   253.5262 │
│ 50e5  │ 20709 │ 6442194.583333333 │   311.0819 │
│ 50d3  │ 20691 │ 6443722.361111111 │   311.4263 │
│ 5025  │ 20517 │ 6444328.888888889 │   314.0970 │
│ 5068  │ 20584 │ 6451205.555555555 │   313.4087 │
└───────┴───────┴───────────────────┴────────────┘

short at 15

...

┏━━━━━━━━━━┳━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━┓
┃ Bytes    ┃ Value      ┃ Expected          ┃ Factor     ┃
┡━━━━━━━━━━╇━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━┩
│ 3303ef16 │ 855895830  │ 5227975.833333333 │     0.0061 │
│ ad130017 │ 2903703575 │ 5230403.611111111 │     0.0018 │
│ 42122e29 │ 1108487721 │ 5236818.055555555 │     0.0047 │
│ 001eb329 │ 2011945    │ 5236838.055555555 │     2.6029 │
│ 453e743f │ 1161720895 │ 6442194.583333333 │     0.0055 │
│ b03d0621 │ 2956789281 │ 6443722.361111111 │     0.0022 │
│ f33f2e21 │ 4081004065 │ 6444328.888888889 │     0.0016 │
│ 572abe3f │ 1462419007 │ 6451205.555555555 │     0.0044 │
└──────────┴────────────┴───────────────────┴────────────┘

uint at 17

┏━━━━━━━━━━┳━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━┓
┃ Bytes    ┃ Value      ┃ Expected          ┃ Factor     ┃
┡━━━━━━━━━━╇━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━┩
│ 03ef1620 │ 66000416   │ 5227975.833333333 │     0.0792 │
│ 13001700 │ 318772992  │ 5230403.611111111 │     0.0164 │
│ 122e2980 │ 305015168  │ 5236818.055555555 │     0.0172 │
│ 1eb32970 │ 515058032  │ 5236838.055555555 │     0.0102 │
│ 3e743f30 │ 1047805744 │ 6442194.583333333 │     0.0061 │
│ 3d062140 │ 1023811904 │ 6443722.361111111 │     0.0063 │
│ 3f2e2100 │ 1059987712 │ 6444328.888888889 │     0.0061 │
│ 2abe3f20 │ 717111072  │ 6451205.555555555 │     0.0090 │
└──────────┴────────────┴───────────────────┴────────────┘

uint at 18

┏━━━━━━━━┳━━━━━━━┳━━━━━━━━━━┳━━━━━━━━┓
┃ Bytes  ┃ Value ┃ Expected ┃ Factor ┃
┡━━━━━━━━╇━━━━━━━╇━━━━━━━━━━╇━━━━━━━━┩
│ ef1620 │ -     │ -        │ -      │
│ 001700 │ -     │ -        │ -      │
│ 2e2980 │ -     │ -        │ -      │
│ b32970 │ -     │ -        │ -      │
│ 743f30 │ -     │ -        │ -      │
│ 062140 │ -     │ -        │ -      │
│ 2e2100 │ -     │ -        │ -      │
│ be3f20 │ -     │ -        │ -      │
└────────┴───────┴──────────┴────────┘

In [27]:
print(f"{0x9150:016b}\n{0x8f50:016b}")

1001000101010000
1000111101010000

In [28]:
packets=get_packets(lambda r,t:isinstance(r,DataFrame) and r.packet.bytes[10]&0xF0==0x90 and r.packet.source==target and r.packet.bytes.find(b'\x06\x00')==11 )
display_packets(packets)

┏━━━━━━━━━━━━━━━━━━━━━┳━━━━━━┳━━━━━━━━━━━━━┳━━━━━━━━━━┳━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━━━┓
┃ Timestamp           ┃ type ┃ type (bits) ┃ From     ┃ To       ┃ Undecoded bytes          ┃
┡━━━━━━━━━━━━━━━━━━━━━╇━━━━━━╇━━━━━━━━━━━━━╇━━━━━━━━━━╇━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━━━┩
│ 2021-03-01 09:55:38 │ 0x29 │ 00101001    │ f03b9ffb │ f0611c3e │ 819006009350027b34581600 │
│ 2021-03-01 11:14:22 │ 0x21 │ 00100001    │ f03b9ffb │ f0611c3e │ 139006009b504bef3c201670 │
│ 2021-03-01 11:30:45 │ 0x29 │ 00101001    │ f03b9ffb │ f0611c3e │ 259006008950cbf032541670 │
│ 2021-03-01 12:14:13 │ 0x29 │ 00101001    │ f03b9ffb │ f0611c3e │ 459006009f50964113111620 │
│ 2021-03-01 12:35:28 │ 0x29 │ 00101001    │ f03b9ffb │ f0611c3e │ e59006009750af5239d21670 │
│ 2021-03-01 12:59:07 │ 0x29 │ 00101001    │ f03b9ffb │ f0611c3e │ 159006009d50d51515761660 │
│ 2021-03-01 13:21:14 │ 0x21 │ 00100001    │ f03b9ffb │ f0611c3e │ af9006009150fa0e0edf1670 │
│ 2021-03-01 13:39:49 │ 0x29 │ 00101001    │ f03b9ffb │ f0611c3e │ 1190060097508be338851690 │
│ 2021-03-01 13:57:45 │ 0x29 │ 00101001    │ f03b9ffb │ f0611c3e │ 279006009b508a5111741670 │
│ 2021-03-01 14:30:46 │ 0x29 │ 00101001    │ f03b9ffb │ f0611c3e │ 399106009150fd3303ef1620 │
│ 2021-03-01 19:14:22 │ 0x29 │ 00101001    │ f03b9ffb │ f0611c3e │ 23900600a350815d16991610 │
│ 2021-03-01 19:28:39 │ 0x29 │ 00101001    │ f03b9ffb │ f0611c3e │ e19006008850e8df1d521600 │
│ 2021-03-01 22:13:25 │ 0x29 │ 00101001    │ f03b9ffb │ f0611c3e │ 3d91060091504cad13001700 │
│ 2021-03-02 01:39:48 │ 0x29 │ 00101001    │ f03b9ffb │ f0bcd31d │ 3990060091507f0200113f10 │
│ 2021-03-02 03:14:12 │ 0x29 │ 00101001    │ f03b9ffb │ 40d0d00a │ 8b900600ee50d8592eeb2560 │
│ 2021-03-02 04:14:13 │ 0x29 │ 00101001    │ f03b9ffb │ 40d0d00a │ e990060093509b2e09dc2500 │
│ 2021-03-02 04:28:31 │ 0x29 │ 00101001    │ f03b9ffb │ 40d0d00a │ 919006007d5085c510d62530 │
│ 2021-03-02 05:52:57 │ 0x29 │ 00101001    │ f03b9ffb │ 40d0d00a │ 8f9006007b500deb1adb2560 │
│ 2021-03-02 07:28:39 │ 0x29 │ 00101001    │ f03b9ffb │ 40d0d00a │ 3590060088501bcb26ab2590 │
│ 2021-03-02 08:14:13 │ 0x29 │ 00101001    │ f03b9ffb │ 40d0d00a │ 9d9006009550632338aa2510 │
│ 2021-03-02 18:06:22 │ 0x29 │ 00101001    │ f03b9ffb │ f091461c │ d59006009a502a0916b12950 │
│ 2021-03-02 19:05:28 │ 0x29 │ 00101001    │ f03b9ffb │ f091461c │ eb9006009850cddb1d9b2930 │
│ 2021-03-02 19:14:22 │ 0x21 │ 00100001    │ f03b9ffb │ f091461c │ 119006009c500748295c2980 │
│ 2021-03-02 19:19:45 │ 0x29 │ 00101001    │ f03b9ffb │ f091461c │ bf9006008850b8f5245c2950 │
│ 2021-03-02 19:28:39 │ 0x29 │ 00101001    │ f03b9ffb │ f091461c │ d39006008850a43b30062940 │
│ 2021-03-02 19:38:05 │ 0x29 │ 00101001    │ f03b9ffb │ f091461c │ 039006008f50e9a1067c2930 │
│ 2021-03-02 19:52:24 │ 0x29 │ 00101001    │ f03b9ffb │ f091461c │ bb900600795078d00e002960 │
│ 2021-03-02 21:16:34 │ 0x29 │ 00101001    │ f03b9ffb │ f091461c │ c991060091505d42122e2980 │
│ 2021-03-02 21:22:43 │ 0x29 │ 00101001    │ f03b9ffb │ f091461c │ 999106008f50b0001eb32970 │
│ 2021-03-03 05:55:11 │ 0x29 │ 00101001    │ f03b9ffb │ f091461c │ af9006009450f58320692970 │
│ 2021-03-03 06:09:28 │ 0x29 │ 00101001    │ f03b9ffb │ f091461c │ 679006008850bffd27282960 │
│ 2021-06-22 12:38:56 │ 0x29 │ 00101001    │ f03b9ffb │ f15df2b3 │ 039106009250e5453e743f30 │
│ 2021-06-22 14:11:17 │ 0x29 │ 00101001    │ f03b9ffb │ 40d0d00a │ a59106009250d3b03d062140 │
│ 2021-06-22 14:44:58 │ 0x29 │ 00101001    │ f03b9ffb │ 40d0d00a │ dd910600925025f33f2e2100 │
│ 2021-06-22 19:56:20 │ 0x29 │ 00101001    │ f03b9ffb │ f1366742 │ 439006007b502d3d0a9f3f70 │
│ 2021-06-22 20:31:18 │ 0x29 │ 00101001    │ f03b9ffb │ f1366742 │ 17910600925068572abe3f20 │
└─────────────────────┴──────┴─────────────┴──────────┴──────────┴──────────────────────────┘

In [29]:
1/7.2

0.1388888888888889

In [30]:
1/0.15

6.666666666666667

In [31]:
37200*7.2

267840.0

In [32]:
642/0x92


4.397260273972603

In [33]:
0x92*7.2

1051.2

In [34]:
df=DataFrame.from_bytes(bytes.fromhex('00FF2AD5001C29F03B9FFBF15DF2B36E9101002532050600BC50E7EC29023F2066D4'))
print(df.packet)

DataPacket Packet contains 26 bytes
┏━━━━━━━━┳━━━━━━━━━━━━━┳━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━┓
┃ Offset ┃ Field Name  ┃ Byte range ┃ Bytes                              ┃ Value            ┃
┡━━━━━━━━╇━━━━━━━━━━━━━╇━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━┩
│   0    │ packet_type │ 0-1        │ 29                                 │ 41               │
│   1    │ destination │ 1-5        │ f03b9ffb                           │ b'\xf0;\x9f\xfb' │
│   5    │ source      │ 5-9        │ f15df2b3                           │ b'\xf1]\xf2\xb3' │
│   9    │ tail        │ 9-None     │ 6e9101002532050600bc50e7ec29023f20 │ None             │
└────────┴─────────────┴────────────┴────────────────────────────────────┴──────────────────┘

In [35]:
df=DataFrame.from_bytes(bytes.fromhex('00FF2AD500172940D0D00AF03B9FFB099106009350A457194F2030CECC'))
print(df.packet)

DataPacket Packet contains 21 bytes
┏━━━━━━━━┳━━━━━━━━━━━━━┳━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━┓
┃ Offset ┃ Field Name  ┃ Byte range ┃ Bytes                    ┃ Value            ┃
┡━━━━━━━━╇━━━━━━━━━━━━━╇━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━┩
│   0    │ packet_type │ 0-1        │ 29                       │ 41               │
│   1    │ destination │ 1-5        │ 40d0d00a                 │ b'@\xd0\xd0\n'   │
│   5    │ source      │ 5-9        │ f03b9ffb                 │ b'\xf0;\x9f\xfb' │
│   9    │ tail        │ 9-None     │ 099106009350a457194f2030 │ None             │
└────────┴─────────────┴────────────┴──────────────────────────┴──────────────────┘

In [38]:
def decode_address(encoded_data):
    int_data=int.from_bytes(encoded_data,'big')
    latMultiplier = (float(2**20)/90)
    lonMultiplier = (float(2**20)/180)
    latNeg = bool(int_data & 0x800000000000)
    lonNeg = bool(int_data & 0x000004000000)

    latEncoded = (int_data & 0x7FFFF8000000) >> 27
    lonEncoded = (int_data & 0x000003FFFFC0) >> 6
    color = (int_data & 0x00000000001F)

    lat=(-1 * (latEncoded / latMultiplier))
    if not latNeg:
        lat += 90

    lon = (lonEncoded / lonMultiplier)
    if (lonNeg):
        lon -=180

    return(lat, lon, color)



In [39]:
from gridstream.protocol.frame import RoutingFrame
from gridstream.protocol.packets.routing import ARPPacket
packets=get_packets(lambda r,t:isinstance(r,RoutingFrame) and isinstance(r.packet,ARPPacket))
print(len(packets))
print(len(get_packets(lambda r,t:isinstance(r,RoutingFrame) and not isinstance(r.packet,ARPPacket))))
locations={}
for f,t in packets:
    address=f.packet.lan_address.hex()
    if address in locations:
        continue
    else:
        locations[address]=decode_address(f.packet.source)
        
print(locations)
    

11742

2

{'f03b9ffb': (33.179569244384766, -96.69891357421875, 0), 'f024177b': (33.17793846130371, -96.69839859008789, 0)}

In [70]:
print(records[0][0].packet)

ARPPacket Packet contains 33 bytes
┏━━━━━━━━┳━━━━━━━━━━━━━┳━━━━━━━━━━━━┳━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┓
┃ Offset ┃ Field Name  ┃ Byte range ┃ Bytes        ┃ Value                       ┃
┡━━━━━━━━╇━━━━━━━━━━━━━╇━━━━━━━━━━━━╇━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┩
│   0    │ packet_type │ 0-1        │ 30           │ 48                          │
│   1    │ destination │ 1-7        │ ffffffffffff │ b'\xff\xff\xff\xff\xff\xff' │
│   7    │ source      │ 7-13       │ 50cfb5d9e400 │ b'P\xcf\xb5\xd9\xe4\x00'    │
│  13    │ sequence    │ 13-14      │ 70           │ 112                         │
│  14    │ uptime      │ 14-18      │ 000c3582     │ 800130                      │
│  18    │ Unknown     │ 18-20      │ a483         │ None                        │
│  20    │ lan_address │ 20-24      │ f03b9ffb     │ b'\xf0;\x9f\xfb'            │
│  24    │ Unknown     │ 24-29      │ 01001e3204   │ None                        │
│  29    │ timing      │ 29-31      │ 14c3         │ 5315                        │
│  31    │ tail        │ 31-None    │ 7e90         │ None                        │
└────────┴─────────────┴────────────┴──────────────┴─────────────────────────────┘

In [75]:
%debug

> <ipython-input-74-d42bc17be6e0>(6)<module>()
      4 locations={}
      5 for f,t in packets:
----> 6     if f.packet.lan_address in locations:
      7         continue
      8     else:



ipdb>  f


ipdb>  f.packet


ipdb>  print(f.packet)


RoutingPacket Packet contains 33 bytes
┏━━━━━━━━┳━━━━━━━━━━━━━┳━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━┓
┃ Offset ┃ Field Name  ┃ Byte range ┃ Bytes                                                            ┃ Value ┃
┡━━━━━━━━╇━━━━━━━━━━━━━╇━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━┩
│   0    │ packet_type │ 0-1        │ 70                                                               │ 112   │
│   1    │ tail        │ 1-None     │ ffffffffffff50d04dd9e4e0b60095bfa7a483f024177b01002432050e797e90 │ None  │
└────────┴─────────────┴────────────┴──────────────────────────────────────────────────────────────────┴───────┘

ipdb>  quit
